In [1]:
import torch
import time
import numpy as np
from model import Graph_OurConvNet
from helper import calculate_avg_accuracy, update_lr, print_results, load_variables

seed = 42
np.random.seed(seed)

if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    torch.cuda.manual_seed(seed)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    torch.manual_seed(seed)

cuda available


In [2]:
def train(net, lr, l2, batch_iters, early_stopping, verbose=False):
    ### optimization parameters
    nb_classes = 7 
    max_iters = 1000
    decay_rate = 1.25
    SAVE_PATH = 'model_state'

    # Optimizer
    optimizer = net.update(lr, l2) 
    t_start = time.time()
    t_start_total = time.time()
    average_loss_old = torch.tensor(1e4).cuda()
    best = running_train_acc = running_train_loss = running_val_loss = 0.0
    tab_results = []

    for iteration in range(1, max_iters):  # loop over the dataset multiple times
        # forward, loss
        net.train()
        pred_y = net.forward(features_x, E_start, E_end, E_identity, E_dropin)
        loss = net.loss(pred_y[idx_train], train_y[idx_train], None) 
        train_acc = calculate_avg_accuracy(nb_classes, train_y[idx_train], pred_y[idx_train]) # training acc
        running_train_acc += train_acc    
        running_train_loss += loss.item()

        # backward, update
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # validation eval
        net.eval()
        y_eval = net.forward(features_x, E_start, E_end, E_identity, E_dropin)
        val_loss = net.loss(y_eval[idx_val], train_y[idx_val], None) 
        running_val_loss += val_loss.item()

        # learning rate, print results
        if not iteration%batch_iters:
            val_accuracy = calculate_avg_accuracy(nb_classes, train_y[idx_val], y_eval[idx_val])
            average_val_loss = running_val_loss/ batch_iters
            avg_train_acc = running_train_acc/ batch_iters

            # update learning rate 
            if val_accuracy < avg_train_acc and avg_train_acc > 0.8:
                optimizer, lr = update_lr(net, optimizer, average_val_loss, average_loss_old, 
                                          lr, decay_rate, early_stopping, verbose)

            # save intermediate results
            if val_accuracy > best:
                torch.save(net.state_dict(), SAVE_PATH)
                best = val_accuracy
            tab_results.append([iteration,average_val_loss,100* val_accuracy, time.time()-t_start_total])

            if verbose:
                print_results(iteration, batch_iters, avg_train_acc, running_train_loss, val_accuracy, lr, t_start)
            if lr < torch.tensor(early_stopping).cuda() and avg_train_acc - val_accuracy > 0.05:
                print("Early Stopping at %d. Highest Val: %.3f " % (iteration, max([tab_results[i][2] for i in range(len(tab_results))])))
                return max([tab_results[i][2] for i in range(len(tab_results))])
                break

            # reset counters
            t_start = time.time()
            running_train_acc = running_train_loss = running_val_loss = 0.0
            average_loss_old = average_val_loss
    return max([tab_results[i][2] for i in range(len(tab_results))])

In [3]:
features_x, train_y, E_start, E_end, E_identity, E_dropin, idx_train, idx_val, idx_test = load_variables()
CUDA = 1
CORA = 1
net_parameters = {}
net_parameters['features'] = features_x.shape[1]
net_parameters['nb_clusters_target'] = torch.max(train_y).item()+1
net_parameters['H'] = 50

### Train

In [ ]:
net_parameters['L'] = 6
net_parameters['Dropout_fc'] = 0.4
net_parameters['Dropout_edge'] = 0.5
net_parameters['Dropout_in'] = 0.0075
lr = 0.001
l2 = 0.005
batch_iters = 20
early_stopping = 50e-6
SAVE_PATH = 'model_state'
test_acc = np.zeros((10,1))


for i in range(1):
    net = Graph_OurConvNet(net_parameters, CORA, CUDA)
    net.cuda()
    
    train(net, lr, l2, batch_iters, early_stopping, verbose=True)
    
    # Eval
    net.load_state_dict(torch.load(SAVE_PATH))
    net.eval()
    y_eval = net.forward(features_x, E_start, E_end, E_identity, E_dropin)

    loss = net.loss(y_eval[idx_test], labels[idx_test], None) 
    accuracy = calculate_avg_accuracy(net_parameters['nb_clusters_target'], 
                                      labels[idx_test], y_eval[idx_test])
    print('\nloss = %.3f, accuracy = %.3f' % (loss.item(), 100* accuracy))
    test_acc[i] += accuracy


iteration= 20, train loss(20iter)= 1.966, lr= 0.0010000, time(20iter)= 22.05
val accuracy= 11.400
train accuracy= 17.036

iteration= 40, train loss(20iter)= 1.884, lr= 0.0010000, time(20iter)= 21.71
val accuracy= 11.400
train accuracy= 25.607

iteration= 60, train loss(20iter)= 1.726, lr= 0.0010000, time(20iter)= 19.89
val accuracy= 17.800
train accuracy= 37.964

iteration= 80, train loss(20iter)= 1.528, lr= 0.0010000, time(20iter)= 19.66
val accuracy= 40.400
train accuracy= 49.036

iteration= 100, train loss(20iter)= 1.351, lr= 0.0010000, time(20iter)= 20.69
val accuracy= 73.000
train accuracy= 56.893

iteration= 120, train loss(20iter)= 1.205, lr= 0.0010000, time(20iter)= 19.96
val accuracy= 74.600
train accuracy= 63.214

iteration= 140, train loss(20iter)= 1.130, lr= 0.0010000, time(20iter)= 20.10
val accuracy= 75.400
train accuracy= 66.321

iteration= 160, train loss(20iter)= 1.075, lr= 0.0010000, time(20iter)= 20.30
val accuracy= 76.400
train accuracy= 68.643

iteration= 180, tra